In [ ]:
%%time
from src.data.make_dataset import load_list, get_stock_data
from src.models.StockModel import StockModel
window_size = 20
start_date = '2017-01-01'
end_date = '2024-09-01'
feature_columns = ['Close', 'Open', 'Volume', 'Range', 'Gap', 'RSI', 'MACD']
target = "Open"

# Load symbols
nasdaq_symbols = load_list("NASDAQ")
sp500_symbols = load_list("SP500")

# Test tickers, sp500 symbols not also in nasdaq
test_tickers = [item for item in sp500_symbols if item not in nasdaq_symbols]
test_tickers = test_tickers[:75]

#tickers = ['^GSPC', '^IXIC', 'AAPL', 'MSFT', 'NVDA', 'AMZN', 'AVGO', 'META', 'GOOGL', 'GOOG', 'MMM', 'ADBE', 'BWA', 'GD', 'IT']
#tickers = ['^GSPC', '^IXIC']
train_tickers = ['^IXIC'] + nasdaq_symbols
#train_tickers = train_tickers[:51]

# Download data
combined_data = get_stock_data(train_tickers, "1d", start_date, end_date)
combined_data.info()
# Test data
test_data = get_stock_data(test_tickers, "1d", start_date, end_date)

layer_config = [(1,32),(1,64),(1,96),(1,128)]
for i in layer_config:
    print(f"XXXXXXXXXXXXXXXX Running {i[0]} {i[1]} layers XXXXXXXXXXXXXXXXXXXX")
    # Create and train model
    stock_model = StockModel(window_size=window_size, feature_columns=feature_columns, target_name=target, export=True)
    
    stock_model.train(combined_data, patience=10, epochs=150, graph=False, layers=i[0], units_per_layer=i[1])
    metrics_dict, metrics_summary = stock_model.evaluate_many(test_data, graph=False)
    print(metrics_dict)
    print(metrics_summary)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 186328 entries, 2017-01-03 00:00:00-05:00 to 2024-08-30 00:00:00-04:00
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Open          186328 non-null  float64
 1   High          186328 non-null  float64
 2   Low           186328 non-null  float64
 3   Close         186328 non-null  float64
 4   Volume        186328 non-null  int64  
 5   Dividends     186328 non-null  float64
 6   Stock Splits  186328 non-null  float64
 7   Ticker        186328 non-null  object 
dtypes: float64(6), int64(1), object(1)
memory usage: 12.8+ MB
XXXXXXXXXXXXXXXX Running 1 32 layers XXXXXXXXXXXXXXXXXXXX
Initializing model:
 - Window size: 20
 - Features: ['Close', 'Open', 'Volume', 'Range', 'Gap', 'RSI', 'MACD']
 - Target: Open
--- Preparing ^IXIC data using 20 window---
Feature columns: ['Close', 'Open', 'Volume', 'Range', 'Gap', 'RSI', 'MACD', 'MACD_Hist']
Target column: Open
Fea

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 32)                  │           5,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,281 (20.63 KB)

 Trainable params: 5,281 (20.63 KB)

 Non-trainable params: 0 (0.00 B)

--- 1/102 Training model for ^IXIC ---
Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0665 - mape: 318.3037 - val_loss: 0.0785 - val_mape: 31.8834
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0539 - mape: 292.9315 - val_loss: 0.0154 - val_mape: 12.6307
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0552 - mape: 304.8530 - val_loss: 0.0081 - val_mape: 8.9336
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0251 - mape: 201.7200 - val_loss: 0.0068 - val_mape: 8.3179
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0087 - mape: 95.5210 - val_loss: 0.0087 - val_mape: 9.9690
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0056 - mape: 69.1840 - val_loss: 0.0090 - val_mape: 10.3626
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0044 - mape: 57.7157 - val_loss: 0.0082 - val_mape: 9.9164
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0038 - mape: 50.8348 - val_loss: 0.0081 - val_mape: 9

In [ ]:
test_tickers = [item for item in sp500_symbols if item not in nasdaq_symbols]
test_tickers = test_tickers[:5]

#tickers = ['^GSPC', '^IXIC', 'AAPL', 'MSFT', 'NVDA', 'AMZN', 'AVGO', 'META', 'GOOGL', 'GOOG', 'MMM', 'ADBE', 'BWA', 'GD', 'IT']
#tickers = ['^GSPC', '^IXIC', '^DJI']
# Test data
test_data = get_stock_data(["^GSPC", "^DJI"], "1d", start_date, end_date)

metrics_dict, mean_metrics = stock_model.evaluate_many(test_data, graph=True)
print(metrics_dict)
print(mean_metrics)
